## TFT
- need to create feature like date and day of week. 

In [1]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import plotly.express as px
import numpy as np


In [2]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [3]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

In [6]:
## proof as data poitns are missing and not all ss_id ahve data for all hours in the dates. 


import pandas as pd

# Define the specific ss_id and the datetime range
ss_id = '6641'
start_datetime = pd.Timestamp('2021-10-27 19:00:00')
end_datetime = pd.Timestamp('2021-10-29 18:59:00')


hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
# Select the data for the specified range and ss_id
selected_data = hourly_pv_data.sel(datetime=slice(start_datetime, end_datetime))

# Check available datetimes
available_datetimes = selected_data['datetime'].values
generation_values = selected_data[ss_id].values

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'datetime': available_datetimes,
    'generation': generation_values
})

print(results_df)


             datetime  generation
0 2021-10-27 19:00:00         0.0
1 2021-10-27 20:00:00         0.0
2 2021-10-27 21:00:00         0.0
3 2021-10-27 22:00:00         0.0
4 2021-10-27 23:00:00         0.0


Conclusion is that not all points  have all hourly data between 2 days, so it is quite possible if i select a point, it might have only next 5 hours, and not all 48 hours. So dataset would not be optimal 25 ss_ids to 50 datetimes and 48 steps. Multiplication won't work, because there are too many missing data points. 

In [7]:

meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


In [8]:
def get_48_hour_range(start_datetime):
    end_datetime = start_datetime + pd.Timedelta(hours=48, minutes=00)
    return start_datetime, end_datetime

def select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours):
    selected_datetimes = []
    available_datetimes = list(datetimes)

    for _ in range(num_selections):
        if not available_datetimes:
            break
        random_datetime = np.random.choice(available_datetimes)
        selected_datetimes.append(random_datetime)
        
        # Remove datetimes within the min_gap_hours range
        available_datetimes = [dt for dt in available_datetimes if dt > random_datetime + pd.Timedelta(hours=min_gap_hours)]

    return selected_datetimes

In [9]:

hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
datetimes = pd.to_datetime(hourly_pv_data['datetime'].values)

data_vars = list(pv_data.data_vars)
pv_sites_id = np.random.choice(data_vars, 50, replace=False)
print(pv_sites_id)

data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : []}

num_selections = 1500
min_gap_hours = 48

for ss_id in pv_sites_id:
    selected_datetimes = select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours)
    
    for start_datetime in selected_datetimes:
        start, end = get_48_hour_range(start_datetime)
        selected_data = hourly_pv_data.sel(datetime=slice(start, end))
        
        for dt, power in zip(selected_data['datetime'].values, selected_data[ss_id].values):
            data_dict['ss_id'].append(int(ss_id))
            data_dict['pv_datetime'].append(dt)
            data_dict['generation'].append(power)
                                           



['7317' '7568' '7268' '10004' '7050' '6516' '6862' '6952' '3435' '26893'
 '26778' '6800' '27057' '8376' '26784' '12917' '9866' '2814' '5826'
 '16920' '7035' '6452' '6670' '6917' '6430' '7276' '2729' '10361' '13670'
 '26981' '26938' '6675' '9367' '12035' '26887' '7295' '11769' '5427'
 '10649' '4035' '9815' '7171' '7339' '8411' '7932' '15765' '6648' '27016'
 '2813' '26822']


In [10]:


pv_df = pd.DataFrame(data_dict)
print(pv_df.shape)
pv_df = pv_df.dropna(subset={'generation'})

pv_df
print(pv_df.shape)


(17046, 3)
(12230, 3)


In [11]:
pv_df.head()

,ss_id,pv_datetime,generation
0,7317,2020-11-16 03:00:00,0.0
1,7317,2020-11-16 04:00:00,0.0
2,7317,2020-11-16 05:00:00,0.0
3,7317,2020-11-16 06:00:00,0.0
4,7317,2020-11-16 07:00:00,0.0


In [12]:
# pv_df.to_csv("Try.csv")

In [14]:
pv_sites_id = [int(id) for id in pv_sites_id]  
# pv_sites_id

In [16]:
pv_site_dict = {'ss_id':[], "lat":[], "long": []}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
    else:
        print('row empty')
        


print(pv_site_dict)
meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
# meta_site_df

{'ss_id': [7317, 7568, 7268, 10004, 7050, 6516, 6862, 6952, 3435, 26893, 26778, 6800, 27057, 8376, 26784, 12917, 9866, 2814, 5826, 16920, 7035, 6452, 6670, 6917, 6430, 7276, 2729, 10361, 13670, 26981, 26938, 6675, 9367, 12035, 26887, 7295, 11769, 5427, 10649, 4035, 9815, 7171, 7339, 8411, 7932, 15765, 6648, 27016, 2813, 26822], 'lat': [55.77, 54.31, 57.11, 53.13, 51.53, 55.55, 53.17, 51.56, 51.39, 51.55, 51.71, 53.14, 51.52, 50.47, 51.24, 53.44, 53.09, 52.14, 57.68, 56.16, 51.42, 53.33, 53.33, 53.1, 51.77, 50.77, 51.61, 55.74, 57.64, 51.32, 51.44, 51.14, 57.21, 55.97, 52.28, 50.34, 54.96, 50.49, 55.93, 52.39, 57.55, 51.45, 55.34, 53.52, 51.29, 51.74, 50.86, 51.59, 52.75, 51.21], 'long': [-4.2, -2.72, -2.19, -1.17, -0.72, -2.84, -0.77, -1.84, 0.01, -1.84, -1.32, -1.17, -0.43, -3.51, -2.33, -3.04, -1.24, -0.3, -2.59, -3.33, -0.29, -2.68, -1.55, -0.17, -1.6, -4.13, -0.24, -4.21, -3.29, -2.44, -1.06, -0.99, -2.29, -3.7, -0.61, -4.79, -1.37, -3.55, -4.16, -1.75, -2.95, -0.89, -1.57, -2.18, 

In [17]:
combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df

,ss_id,pv_datetime,generation,lat,long
0,7317,2020-11-16 03:00:00,0.0,55.77,-4.20
1,7317,2020-11-16 04:00:00,0.0,55.77,-4.20
2,7317,2020-11-16 05:00:00,0.0,55.77,-4.20
3,7317,2020-11-16 06:00:00,0.0,55.77,-4.20
4,7317,2020-11-16 07:00:00,0.0,55.77,-4.20
...,...,...,...,...,...
12225,26822,2021-10-27 19:00:00,0.0,51.21,-2.62
12226,26822,2021-10-27 20:00:00,0.0,51.21,-2.62
12227,26822,2021-10-27 21:00:00,0.0,51.21,-2.62
12228,26822,2021-10-27 22:00:00,0.0,51.21,-2.62


In [19]:
combined_df['pv_datetime'] = pd.to_datetime(combined_df['pv_datetime'])

# Extract date and hour from pv_datetime
combined_df['pv_date'] = combined_df['pv_datetime'].dt.date
combined_df['pv_hour'] = combined_df['pv_datetime'].dt.hour
combined_df

,ss_id,pv_datetime,generation,lat,long,pv_date,pv_hour
0,7317,2020-11-16 03:00:00,0.0,55.77,-4.20,2020-11-16,3
1,7317,2020-11-16 04:00:00,0.0,55.77,-4.20,2020-11-16,4
2,7317,2020-11-16 05:00:00,0.0,55.77,-4.20,2020-11-16,5
3,7317,2020-11-16 06:00:00,0.0,55.77,-4.20,2020-11-16,6
4,7317,2020-11-16 07:00:00,0.0,55.77,-4.20,2020-11-16,7
...,...,...,...,...,...,...,...
12225,26822,2021-10-27 19:00:00,0.0,51.21,-2.62,2021-10-27,19
12226,26822,2021-10-27 20:00:00,0.0,51.21,-2.62,2021-10-27,20
12227,26822,2021-10-27 21:00:00,0.0,51.21,-2.62,2021-10-27,21
12228,26822,2021-10-27 22:00:00,0.0,51.21,-2.62,2021-10-27,22


## Matching pv data with NWP
Need to match pv_datetime, with nwp, split pv_datetime to date and time seperatly, match pv_date with nwp init_time, and pv_hour with nwp steps. 

- 12 is midnight, so need to match with previous time.  using `backfill`

In [41]:
dataframes = []

In [42]:
dataframes = []

for index, row in combined_df.iterrows():

    ss_id = row['ss_id']

    # Match pv_date to init_time
    init_time_sel = nwp_data.sel(init_time=row['pv_datetime'], method="backfill")
    
    # Match pv_hour to step
    step_sel = init_time_sel.sel(step=pd.Timedelta(hours=row['pv_hour']), method="nearest")
    
    data_sel = step_sel.sel(latitude=row['lat'], method="nearest") \
                       .sel(longitude=row['long'], method="nearest")
    

    data_df = data_sel.to_dataframe().reset_index()
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

    pivot_df['ss_id'] = ss_id
    pivot_df['pv_datetime'] = row['pv_datetime']
    pivot_df['generation'] = row['generation']
    pivot_df['pv_hour'] = row['pv_hour']

    dataframes.append(pivot_df)

In [45]:
# dataframes
print(len(dataframes))

12230


In [46]:
final_df = pd.concat(dataframes, ignore_index=True)
desired_order = ['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100']
final_df = final_df[desired_order]
print(final_df.shape)

(12230, 20)


In [47]:
final_df

variable,ss_id,init_time,step,generation,pv_datetime,pv_hour,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100
0,7317,2020-11-16 12:00:00,0 days 03:00:00,0.0,2020-11-16 03:00:00,3,3655011.0,4.034346e+05,48575.207031,0.790100,0.480713,0.884857,0.0,22400.0,281.239746,0.977741,3.625758,6.060360,4.602473,7.247358
1,7317,2020-11-16 12:00:00,0 days 04:00:00,0.0,2020-11-16 04:00:00,4,4845682.0,4.393022e+05,53559.214844,0.432098,0.934845,0.908295,0.0,73472.0,281.080566,0.972164,2.412020,4.402522,4.528537,7.413259
2,7317,2020-11-16 12:00:00,0 days 05:00:00,0.0,2020-11-16 05:00:00,5,6047190.0,4.394795e+05,53557.402344,0.385406,0.970459,0.995361,0.0,73856.0,281.265381,0.998629,1.667396,3.254663,4.643868,7.920271
3,7317,2020-11-16 12:00:00,0 days 06:00:00,0.0,2020-11-16 06:00:00,6,7292510.0,4.394795e+05,53557.402344,0.786957,0.231201,0.902100,0.0,73856.0,281.935547,0.976267,1.377057,2.618967,5.833754,9.853894
4,7317,2020-11-16 12:00:00,0 days 07:00:00,0.0,2020-11-16 07:00:00,7,8533754.0,4.394795e+05,53557.402344,0.916443,0.716919,0.999725,0.0,73856.0,282.482666,0.999971,1.860905,3.472870,5.542457,9.472516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12225,26822,2021-10-28 00:00:00,0 days 19:00:00,0.0,2021-10-27 19:00:00,19,24054816.0,5.793461e+06,678925.000000,0.989990,1.000000,1.000000,0.0,7454208.0,287.276611,1.000000,-1.377494,-1.745785,7.361919,11.765965
12226,26822,2021-10-28 00:00:00,0 days 20:00:00,0.0,2021-10-27 20:00:00,20,25389312.0,5.793461e+06,678925.000000,0.999969,1.000000,0.975342,0.0,7454208.0,287.119385,1.000000,-2.175486,-3.005329,7.762441,12.470768
12227,26822,2021-10-28 00:00:00,0 days 21:00:00,0.0,2021-10-27 21:00:00,21,26734528.0,5.793752e+06,678925.000000,1.000000,1.000000,1.000000,0.0,7454720.0,287.091553,1.000000,-1.461416,-1.617608,8.753415,13.863843
12228,26822,2021-10-28 00:00:00,0 days 22:00:00,0.0,2021-10-27 22:00:00,22,28090224.0,5.793752e+06,678941.687500,0.998291,1.000000,0.584778,0.0,7454720.0,287.500244,1.000000,-0.735680,-0.549637,9.790599,15.084077


In [48]:
final_df.to_csv("result_data/final_data_tft.csv")